In [ ]:
import pandas as pd
import re
import requests
import pymssql
from bs4 import BeautifulSoup

In [ ]:
#################### Single Look and Exploration #########################################


#print(soup.prettify())

#results = soup.findAll('div', {'class':'lister-item-content'})
#result = results[0]
#title = result.findAll('a')[0].text
#year = result.findAll('span')[1].text


results = soup.findAll('div', {'class':'lister-item mode-detail'})
movie = results[0]

title = movie.find('h3').a.text
release_year = int(re.search("\d\d\d\d", movie.find('h3').findAll('span')[1].text).group(0))
age_rating = movie.find('p', {'class': 'text-muted text-small'}).find('span').text
runtime = int(re.search("\d*", movie.find('p', {'class': 'text-muted text-small'}).find('span', {'class':'runtime'}).text).group(0))
genres = movie.find('p', {'class': 'text-muted text-small'}).find('span', {'class':'genre'}).text
rating = float(movie.find('div', {'class': 'ipl-rating-widget'}).find('span', {'class':'ipl-rating-star__rating'}).text)
metascore = int(movie.find('div', {'class':'inline-block ratings-metascore'}).span.text)
director = movie.findAll('p')[2].a.text
votes = movie.findAll('p')[3].find('span', {'name':'nv'}).text
gross = int(movie.findAll('p')[3].findAll('span', {'name':'nv'})[1].get('data-value').replace(",",''))

In [ ]:
#################### Scale the prior setup ########################################
def itJustWorks(TotalPages):
# Input: Function receives the number of sequential URL pages as a parameter 

# Output: a list of dictionaries with attribute names as a key: attribute values as value; this setup is then easily converted into a pandas DataFrae object

# Initialize list to hold dictionaries
    df = []

    for pageNum in range(TotalPages):
        pageurl = 'https://www.imdb.com/list/ls021161997/?st_dt=&mode=detail&page={0}&sort=num_votes,desc'.format(pageNum + 1)
        page = requests.get(pageurl)
        
        if (page.status_code != 200):
            raise NameError("Unsuccessful scraping of page " + pageNum)

        soup = BeautifulSoup(page.text, 'html.parser')
        results = soup.findAll('div', {'class':'lister-item mode-detail'})


        for i in range(0, len(results)):
            movie = results[i]
            data = {
                'title' : movie.find('h3').a.text,
                'release_year' : int(re.search("\d\d\d\d", movie.find('h3').findAll('span')[1].text).group(0)),
                'age_rating' : movie.find('p', {'class': 'text-muted text-small'}).find('span').text,
                'runtime' : int(re.search("\d*", movie.find('p', {'class': 'text-muted text-small'}).find('span', {'class':'runtime'}).text).group(0)),
                'rating' : float(movie.find('div', {'class': 'ipl-rating-widget'}).find('span', {'class':'ipl-rating-star__rating'}).text),
                'director' : movie.findAll('p')[2].a.text,
                'votes' : int(movie.findAll('p')[3].find('span', {'name':'nv'}).text.replace(",",'')),
            }
            print(data['title'])
            try:
                data['metascore'] = int(movie.find('div', {'class':'inline-block ratings-metascore'}).span.text)
            except AttributeError:
                data['metascore'] = -1
            try:
                data['gross'] = int(movie.findAll('p')[3].findAll('span', {'name':'nv'})[1].get('data-value').replace(",",''))
            except (AttributeError, IndexError):
                data['gross'] = -1
        
        
            df.append(data)
    df = pd.DataFrame(df, columns = ['title', 'release_year', 'age_rating', 'runtime','rating','metascore', 'director', 'votes', 'gross'])
    
    return(df)

In [ ]:
## Run prior function through all pages of the imdb website (placed 10 rather than the value from the webpage html because the latter didn't exist. 
# They relied on directional arrows to guide the user through the list.)

df = itJustWorks(10)
print(df)
df.to_csv(path_or_buf='movies.csv')

In [ ]:
## Connect to Azure DB

conn = pymssql.connect(server, username, password, database)
cursor = conn.cursor()

In [ ]:
### Insert From File and Close Connection

# Drop index attribute (not needed)
df = pd.read_csv('movies.csv').drop('Unnamed: 0', axis = 1)

# Get rid of title and author name apostrophies (the insertion code below cannot work with double quotes; this cost me so much time)
df['title'] = df['title'].str.replace("\'", "")
df['director'] = df['director'].str.replace("\'", "")

for index, row in df.iterrows():

    cursor.execute("INSERT INTO dbo.nmarangi_spookymovies (title, release_year, age_rating, runtime, rating, metascore, director, votes, gross_profit) values(%s,%d,%s,%d,%f,%d,%s,%d,%d)" % ("\'" + row.title + "\'", row.release_year, "\'" + row.age_rating + "\'", row.runtime, row.rating, row.metascore, "\'" + row.director + "\'", row.votes, row.gross))
    print(index)
conn.commit()
conn.close()